In [ ]:
# | default_exp create_vegetation_parameters

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *
from fastcore.utils import *

In [ ]:
# | export
import os
import math
import warnings
import numpy as np
from pathlib import Path
from typing import Dict, List
from sureau_ecos_py.soil_utils import (
    compute_theta_at_given_p_soil,
    compute_theta_at_given_p_soil_camp,
)
from sureau_ecos_py.plant_utils import (
    read_vegetation_file,
    convert_f_cm3_to_v_mm,
    distribute_conductances,
)

from sureau_ecos_py.create_soil_parameters import create_soil_parameters
from sureau_ecos_py.create_modeling_options import create_modeling_options
from sureau_ecos_py.create_stand_parameters import create_stand_parameters

In [ ]:
# | export


def create_vegetation_parameters(
    stand_parameters: Dict,  # Dictionary created using the `create_stand_parameters` function
    soil_parameters: Dict,  # Dictionary created using the `create_soil_parameters` function
    file_path: Path = None,  # Path to a csv file trait parameters. Check `read_vegetation_file` for more information i.e path/to/parameter_values.csv
    list_of_parameters: List = None,  # A list containing the necessary input parameters instead of reading them in file. Will only be used if 'file_path' arguement is not provided
    modeling_options: Dict = None,  # Dictionary created using the `create_modeling_options` function
    sep: str = ";",  # CSV file separator can be ',' or ';'
) -> Dict:  # Dictionary containing parameters to run the model
    "Create a dictionary with the vegetation parameters to run \code{run.SurEauR}"

    # Assert parameters ---------------------------------------------------------

    # Make sure the file_path exist or is None
    assert file_path is None or os.path.exists(
        file_path
    ), f"Path: {file_path} not found, check spelling or set file_path = None"

    # Raise error if file_path and list_of_parameters both are missing
    if file_path is None and list_of_parameters is None:
        raise ValueError(
            "Both file_path and list_of_parameters are missing, provide one of them"
        )

    # Raise error if file_path and list_of_parameters both are provided
    if file_path is not None and list_of_parameters is not None:
        raise ValueError(
            "Both file_path and list_of_parameters are provided, only one of these two arguments should be used"
        )

    # Delete as soon the option is implemented
    if list_of_parameters is not None:
        raise ValueError(
            "Option for including list_of_parameters not implemented yet"
        )

    # Make sure that modeling_options is a dictionary
    assert isinstance(
        modeling_options, Dict
    ), f"modeling_options must be a dictionary not a {type(modeling_options)}"

    # Make sure that stand_parameters is a dictionary
    assert isinstance(
        stand_parameters, Dict
    ), f"stand_parameters must be a dictionary not a {type(stand_parameters)}"

    # Make sure that soil_parameters is a dictionary
    assert isinstance(
        soil_parameters, Dict
    ), f"soil_parameters must be a dictionary not a {type(soil_parameters)}"

    # Create vegetation_parameters from function inputs -------------------------

    # Read CSV file
    vegetation_parameters = read_vegetation_file(
        file_path=file_path, modeling_options=modeling_options, sep=sep
    )

    # Compute stomatal response parameters for the sigmoid from P12_gs and P88_gs
    # provided in the CSV file
    if (
        modeling_options["stomatal_reg_formulation"] == "sigmoid"
        and "p50_gs" not in vegetation_parameters
    ):
        # Calculate p50_gs
        vegetation_parameters["p50_gs"] = (
            vegetation_parameters["p88_gs"] + vegetation_parameters["p12_gs"]
        ) / 2
        vegetation_parameters["slope_gs"] = 100 / (
            vegetation_parameters["p12_gs"] - vegetation_parameters["p88_gs"]
        )

    else:
        print("p50_gs already provided")

    # Get maximum leaf area index of the stand (LAImax) from stand parameters
    vegetation_parameters["lai_max"] = stand_parameters["lai_max"]

    # Calculate root distribution within each soil layer (Jackson et al. 1996)
    warnings.warn("Make sure that depth from soil_parameters is in meters")

    # Create numpy array
    vegetation_parameters["root_distribution"] = np.array([0, 0, 0], dtype=float)

    # soil_parameters["depth"][n] is multiplied by 100 to convert it from meters
    # to centimeters

    # Layer 1
    vegetation_parameters["root_distribution"][0] = 1 - (
        vegetation_parameters["betarootprofile"]
        ** (soil_parameters["soil_depths"][0] * 100)
    )

    # Layer 2
    vegetation_parameters["root_distribution"][1] = (
        1
        - vegetation_parameters["betarootprofile"]
        ** (soil_parameters["soil_depths"][1] * 100)
    ) - vegetation_parameters["root_distribution"][0]

    # Layer 3
    vegetation_parameters["root_distribution"][2] = 1 - (
        vegetation_parameters["root_distribution"][0]
        + vegetation_parameters["root_distribution"][1]
    )

    # Calculate turgor loss point (tlp)
    tlp = (
        vegetation_parameters["pifullturgor_leaf"]
        * vegetation_parameters["epsilonsym_leaf"]
    ) / (
        vegetation_parameters["pifullturgor_leaf"]
        + vegetation_parameters["epsilonsym_leaf"]
    )

    # Pedotransfer function == vg
    if modeling_options["pedo_transfer_formulation"] == "vg":
        theta_at_tlp = compute_theta_at_given_p_soil(
            psi_target=np.abs(tlp),
            theta_res=soil_parameters["residual_capacity_vg"],
            theta_sat=soil_parameters["saturation_capacity_vg"],
            alpha_vg=soil_parameters["alpha_vg"],
            n_vg=soil_parameters["n_vg"],
        )

        theta_at_p50 = compute_theta_at_given_p_soil(
            psi_target=np.abs(vegetation_parameters["p50_vc_leaf"]),
            theta_res=soil_parameters["residual_capacity_vg"],
            theta_sat=soil_parameters["saturation_capacity_vg"],
            alpha_vg=soil_parameters["alpha_vg"],
            n_vg=soil_parameters["n_vg"],
        )

        # Compute TAW @Tlp @P12 & @P50 (Diagnoostic)
        vegetation_parameters["taw_at_tlp"] = np.sum(
            soil_parameters["v_field_capacity"]
            - convert_f_cm3_to_v_mm(
                x=theta_at_tlp,
                rock_fragment_content=soil_parameters["rock_fragment_content"],
                layer_thickness=soil_parameters["layer_thickness"],
            )
        )

        vegetation_parameters["taw_at_p50"] = np.sum(
            soil_parameters["v_field_capacity"]
            - convert_f_cm3_to_v_mm(
                x=theta_at_p50,
                rock_fragment_content=soil_parameters["rock_fragment_content"],
                layer_thickness=soil_parameters["layer_thickness"],
            )
        )

        print(
            f"Available water capacity @Tlp (VG):{vegetation_parameters['taw_at_tlp']} mm"
        )
        print(
            f"Available water capacity @P50 (VG):{vegetation_parameters['taw_at_p50']} mm"
        )

    # Pedotransfer function == Campbell
    elif modeling_options["pedo_transfer_formulation"] == "campbell":
        theta_at_tlp = compute_theta_at_given_p_soil_camp(
            psi_target=tlp,
            theta_sat=soil_parameters["saturation_capacity_campbell"],
            psie=soil_parameters["psie"],
            b_camp=soil_parameters["b_campbell"],
        )

        theta_at_p50 = compute_theta_at_given_p_soil_camp(
            psi_target=vegetation_parameters["p50_vc_leaf"],
            theta_sat=soil_parameters["saturation_capacity_campbell"],
            psie=soil_parameters["psie"],
            b_camp=soil_parameters["b_campbell"],
        )

        # Compute TAW @Tlp @P12 & @P50 (Diagnoostic)
        vegetation_parameters["taw_at_tlp"] = np.sum(
            soil_parameters["v_field_capacity"]
            - convert_f_cm3_to_v_mm(
                x=theta_at_tlp,
                rock_fragment_content=soil_parameters["rock_fragment_content"],
                layer_thickness=soil_parameters["layer_thickness"],
            )
        )

        vegetation_parameters["taw_at_p50"] = np.sum(
            soil_parameters["v_field_capacity"]
            - convert_f_cm3_to_v_mm(
                x=theta_at_p50,
                rock_fragment_content=soil_parameters["rock_fragment_content"],
                layer_thickness=soil_parameters["layer_thickness"],
            )
        )

        print(
            f"Available water capacity @Tlp (Campbell):{vegetation_parameters['taw_at_tlp']} mm"
        )
        print(
            f"Available water capacity @P50 (Campbell):{vegetation_parameters['taw_at_p50']} mm"
        )

    else:
        raise ValueError("pedo_transfer_formulation is missing")

    # Determine root lenght (La gardner cowan)

    # Total fine root area equation 18 from the SurEau-Ecos paper
    rai = vegetation_parameters["lai_max"] * vegetation_parameters["froottoleaf"]

    # Root length per soil area
    vegetation_parameters["la"] = (
        rai * vegetation_parameters["root_distribution"]
    ) / (2 * math.pi * vegetation_parameters["rootradius"])

    # Root length per soil volume
    vegetation_parameters["lv"] = vegetation_parameters["la"] / (
        soil_parameters["layer_thickness"]
        * (1 - (soil_parameters["rock_fragment_content"] / 100))
    )

    # Calculate the different conductance of the plant from k_PlantInit
    conductances = distribute_conductances(
        k_plant_init=vegetation_parameters["k_plantinit"],
        ri=vegetation_parameters["root_distribution"],
        frac_leaf_sym=vegetation_parameters["frac_leaf_sym"],
    )

    # I am assumming that this is the Maximum conductance from trunk apoplasm to
    # the leaf apoplasm KSApo−LApo,max
    vegetation_parameters["k_slapo_init"] = conductances["k_slapo_init"]

    # I am assumming that this is the Maximum conductance from the root surface
    # to the stem apoplasm
    vegetation_parameters["k_rsapo_init"] = conductances["k_rsapo_init"]

    # I am assumming that this is the Conductance from the leaf apoplasm to leaf
    # symplasm
    vegetation_parameters["k_lsym_init"] = conductances["k_lsym_init"]

    # Add Hydaulic conductance of the plant from soil to leaves
    vegetation_parameters["k_plant_init"] = conductances["k_plant_init"]

    # Add Volume of tissue of the stem (includes the root, trunk and branches)
    # Original line is adding it twice. not added
    # TTT$vol_Stem = TTT$vol_Stem
    # vegetation_parameters['vol_stem'] = vegetation_parameters['vol_stem']

    return vegetation_parameters

#### __Example: Create vegetation parameters dictionary__

__List of traits required:__

<table>
  <tr>
   <td><strong>term_in_code</strong>
   </td>
   <td><strong>term_in_paper</strong>
   </td>
   <td><strong>Definition</strong>
   </td>
   <td><strong>Units</strong>
   </td>
  </tr>
  <tr>
   <td>P50_VC_Leaf
   </td>
   <td>ψ50,L
   </td>
   <td>Water potential causing 50% Cavitation in the vulnerability curve
   </td>
   <td>MPa
   </td>
  </tr>
  <tr>
   <td>slope_VC_Leaf
   </td>
   <td>slopeL
   </td>
   <td>Slope of rate of leaf embolism spread at ψ50,L
   </td>
   <td>%/MPa
   </td>
  </tr>
  <tr>
   <td>epsilonSym_Leaf
   </td>
   <td>εL
   </td>
   <td>Modulus of elasticity of the leaf symplasm
   </td>
   <td>MPa
   </td>
  </tr>
  <tr>
   <td>PiFullTurgor_Leaf
   </td>
   <td>π0L
   </td>
   <td>Osmotic potential at full turgor of the leaf symplasm
   </td>
   <td>MPa
   </td>
  </tr>
  <tr>
   <td>apoFrac_Leaf
   </td>
   <td>αLApo
   </td>
   <td>Apoplasmic Fraction in leaves/Leaf apoplasmic fraction
   </td>
   <td>Unitless
   </td>
  </tr>
  <tr>
   <td>LDMC
   </td>
   <td>LDMC
   </td>
   <td>Leaf dry matter content (dry mass over saturated mass)
   </td>
   <td>mgMS/g??
   </td>
  </tr>
  <tr>
   <td>LMA
   </td>
   <td>LMA
   </td>
   <td>Leaf mass per area
   </td>
   <td>g/m2leaf
   </td>
  </tr>
  <tr>
   <td>K
   </td>
   <td>k
   </td>
   <td>Light extinction coefficient of the vegetation layer
   </td>
   <td>unitless
   </td>
  </tr>
  <tr>
   <td>k_PlantInit
   </td>
   <td><strong>not found</strong>
   </td>
   <td>Hydaulic conductance of the plant from soil to leaves
   </td>
   <td>mmol/MPa/s/m2leaf?
   </td>
  </tr>
  <tr>
   <td>gmin20
   </td>
   <td>gcuti20
   </td>
   <td>Minimum conductance (gmin) at the reference temperature.
   </td>
   <td>mmol/m2leaf/s-1
   </td>
  </tr>
  <tr>
   <td>TPhase_gmin
   </td>
   <td>TPhase
   </td>
   <td>Temperature for phase transition of minimum conductance/Temperature for transition phase of gcuti
   </td>
   <td>degC
   </td>
  </tr>
  <tr>
   <td>Q10_1_gmin
   </td>
   <td>Q10a
   </td>
   <td>Q10 value for gmin = f(T) &lt;= Tphase_gmin/ Temperature dependance of gcuti when T ≤ TPhase
   </td>
   <td>unitless
   </td>
  </tr>
  <tr>
   <td>Q10_2_gmin
   </td>
   <td>Q10b
   </td>
   <td>Q10 value for gmin = f(T) > Tphase_gmin/ Temperature dependance of gcuti when T > TPhase
   </td>
   <td>unitless
   </td>
  </tr>
  <tr>
   <td>gmin_S
   </td>
   <td><strong>not found</strong>
   </td>
   <td>conductance (gmin) of the stem
   </td>
   <td>Unitless??
   </td>
  </tr>
  <tr>
   <td>canopyStorageParam
   </td>
   <td>cws
   </td>
   <td>Depth of water that can be retained by leaves and trunks per unit of leaf area index (used to compute the canopy water storage capacity as a function of LAI
   </td>
   <td>l/m2leaf
   </td>
  </tr>
  <tr>
   <td>rootRadius
   </td>
   <td>dR
   </td>
   <td>radius of roots
   </td>
   <td>meters
   </td>
  </tr>
  <tr>
   <td>fRootToLeaf
   </td>
   <td>RaLa
   </td>
   <td>Root-to-leaf area ratio
   </td>
   <td>unitless
   </td>
  </tr>
  <tr>
   <td>betaRootProfile
   </td>
   <td>β
   </td>
   <td>parameter for the distribution of roots in the soil
   </td>
   <td>unitless
   </td>
  </tr>
  <tr>
   <td>PiFullTurgor_Stem
   </td>
   <td>π0S
   </td>
   <td>Osmotic potential at full turgor of the stem symplasm
   </td>
   <td>MPa
   </td>
  </tr>
  <tr>
   <td>epsilonSym_Stem
   </td>
   <td>εS
   </td>
   <td>Modulus of elasticity of the stem symplasm
   </td>
   <td>MPa
   </td>
  </tr>
  <tr>
   <td>apoFrac_Stem
   </td>
   <td>αSApo
   </td>
   <td>Stem apoplasmic fraction of the wood water volume
   </td>
   <td>Unitless
   </td>
  </tr>
  <tr>
   <td>symFrac_Stem
   </td>
   <td>αSSym
   </td>
   <td>Stem symplasmic fraction of the wood water volume
   </td>
   <td>Unitless
   </td>
  </tr>
  <tr>
   <td>vol_Stem
   </td>
   <td>VS
   </td>
   <td>Volume of tissue of the stem (includes the root, trunk and branches)
   </td>
   <td>L m−2 soil
   </td>
  </tr>
  <tr>
   <td>fTRBToLeaf
   </td>
   <td><strong>not found</strong>
   </td>
   <td>Definition <strong>not found</strong>
   </td>
   <td>
   </td>
  </tr>
  <tr>
   <td>C_LApoInit
   </td>
   <td>CLApo
   </td>
   <td>Capacitance of the leaf apoplasm
   </td>
   <td>mmol m−2 leaf MPa−1
   </td>
  </tr>
  <tr>
   <td>C_SApoInit
   </td>
   <td>CSApo
   </td>
   <td>Capacitance of the stem apoplasm
   </td>
   <td>mmol m−2 leaf MPa−1
   </td>
  </tr>
  <tr>
   <td>P50_VC_Stem
   </td>
   <td>ψ50,S
   </td>
   <td>Water potential causing 50 % loss of stem hydraulic conductance
   </td>
   <td>MPa
   </td>
  </tr>
  <tr>
   <td>slope_VC_Stem
   </td>
   <td>slopeS
   </td>
   <td>Slope of rate of stem embolism spread at ψ50,S
   </td>
   <td>% MPa−1
   </td>
  </tr>
  <tr>
   <td>k_SSymInit
   </td>
   <td><strong>not found</strong>
   </td>
   <td>Definition <strong>not found</strong>
   </td>
   <td><strong>not found</strong>
   </td>
  </tr>
  <tr>
   <td>foliage
   </td>
   <td><strong>not found</strong>
   </td>
   <td>Definition <strong>not found</strong>
   </td>
   <td>Not applicable
   </td>
  </tr>
  <tr>
   <td><strong>not found</strong>
   </td>
   <td>t0
   </td>
   <td>Initial date of the forcing period for leaf phenology
   </td>
   <td>DOY
   </td>
  </tr>
  <tr>
   <td><strong>not found</strong>
   </td>
   <td>TD
   </td>
   <td>Minimum temperature to start cumulating temperature for budburst
   </td>
   <td>◦C
   </td>
  </tr>
  <tr>
   <td><strong>not found</strong>
   </td>
   <td>F∗
   </td>
   <td>Amount of forcing temperature to reach budburst
   </td>
   <td>◦C
   </td>
  </tr>
  <tr>
   <td>RAI
   </td>
   <td>RLAI
   </td>
   <td>LAI growth rate per day
   </td>
   <td>LAI d−1
   </td>
  </tr>
  <tr>
   <td>k_RSApoInit
   </td>
   <td>KR-SApo,max
   </td>
   <td>Maximum conductance from the root surface to the stem apoplasm
   </td>
   <td>mmol m−2 leaf s −1 MPa−1
   </td>
  </tr>
  <tr>
   <td>k_SLApoInit
   </td>
   <td>KSApo−LApo,max
   </td>
   <td>Maximum conductance from trunk apoplasm to the leaf apoplasm
   </td>
   <td>mmol m−2 leaf s −1 MPa−1
   </td>
  </tr>
  <tr>
   <td><strong>not found</strong>
   </td>
   <td>KSSym
   </td>
   <td>Conductance from the stem apoplasm to stem symplasm
   </td>
   <td>mmol m−2 leaf s −1 MPa−1
   </td>
  </tr>
  <tr>
   <td>k_LSymInit
   </td>
   <td>KLSym
   </td>
   <td>Conductance from the leaf apoplasm to leaf symplasm
   </td>
   <td>mmol m−2 leaf s −1 MPa−1
   </td>
  </tr>
  <tr>
   <td><strong>not found</strong>
   </td>
   <td>Succulence
   </td>
   <td>Leaf succulence (water content per unit of leaf area)
   </td>
   <td>g m−2 leaf
   </td>
  </tr>
  <tr>
   <td>P50_gs
   </td>
   <td>ψgs50
   </td>
   <td>Water potential causing 50 % stomatal closure
   </td>
   <td>MPa
   </td>
  </tr>
  <tr>
   <td><strong>not found</strong>
   </td>
   <td>slopegs
   </td>
   <td>Rate of decrease in stomatal conductance at ψgs,50
   </td>
   <td>% MPa−1
   </td>
  </tr>
  <tr>
   <td><strong>not found</strong>
   </td>
   <td>gstom_min
   </td>
   <td>Minimum stomatal conductance
   </td>
   <td>mmol m−2 leaf s −1
   </td>
  </tr>
  <tr>
   <td><strong>not found</strong>
   </td>
   <td>gstom_max
   </td>
   <td>Maximum stomatal conductance
   </td>
   <td>mmol m−2 leaf s −1
   </td>
  </tr>
  <tr>
   <td><strong>not found</strong>
   </td>
   <td>δ
   </td>
   <td>Response of gstom to light
   </td>
   <td>unitless
   </td>
  </tr>
  <tr>
   <td><strong>not found</strong>
   </td>
   <td>Toptim
   </td>
   <td>Temperature at maximal stomatal conductance
   </td>
   <td>◦C
   </td>
  </tr>
  <tr>
   <td><strong>not found</strong>
   </td>
   <td>Tsens
   </td>
   <td>Stomatal sensitivity to temperature
   </td>
   <td>◦C
   </td>
  </tr>
  <tr>
   <td><strong>not found</strong>
   </td>
   <td>θs
   </td>
   <td>Soil water content at saturation
   </td>
   <td>unitless
   </td>
  </tr>
  <tr>
   <td><strong>not found</strong>
   </td>
   <td>θr
   </td>
   <td>Residual soil water content
   </td>
   <td>unitless
   </td>
  </tr>
  <tr>
   <td><strong>not found</strong>
   </td>
   <td>α
   </td>
   <td>Inverse of the air entry potential
   </td>
   <td>1/MPa
   </td>
  </tr>
  <tr>
   <td><strong>not found</strong>
   </td>
   <td>n
   </td>
   <td>Pore size distribution index
   </td>
   <td>unitless
   </td>
  </tr>
  <tr>
   <td><strong>not found</strong>
   </td>
   <td>I
   </td>
   <td>Shape parameter for the Van Genuchten equation
   </td>
   <td>unitless
   </td>
  </tr>
  <tr>
   <td><strong>not found</strong>
   </td>
   <td>ksat
   </td>
   <td>Soil hydraulic conductivity at saturation
   </td>
   <td>mmol m−1 soil s −1 MPa−1
   </td>
  </tr>
  <tr>
   <td><strong>not found</strong>
   </td>
   <td>gsoil0
   </td>
   <td>Reference soil conductance to water vapor
   </td>
   <td>mmol m−2 soil s −1
   </td>
  </tr>
</table>

In [ ]:
modeling_options_dict = create_modeling_options(
    time_step_for_evapo=2,
    reset_swc=True,
    avoid_water_soil_transfer=True,
    constant_climate=False,
    defoliation=True,
    soil_evapo=True,
    threshold_mortality=51,
    etp_formulation="pt",
    rn_formulation="linear",
    comp_options_for_evapo="custom",
    stomatal_reg_formulation="sigmoid",
    transpiration_model="jarvis",
    numerical_scheme="implicit",
    pedo_transfer_formulation="campbell",
)

In [ ]:
stand_params = create_stand_parameters(
    file_path="./sample_data/stand_example.csv",
    lai_max=None,
    latitude=None,
    longitude=None,
    sep=";",
)

In [ ]:
soil_params = create_soil_parameters(
    file_path="./sample_data/soil_example.csv",
    modeling_options=modeling_options_dict,
    default_soil=False,
    offset_psoil=1,
    psoil_at_field_capacity=-1,
)

There is an offset on Psoil of 1 MPa
Psoil at field capacity = -0.001 MPa
You are using campbell pedotransfer formulation
Available water capacity Wilting: 273.74477742718403 mm
Available water capacity Residual: 307.81195119276424 mm
Can soil_params["v_soil_storage_capacity"] be negative?? Ask


#### Traits required:


In [ ]:
create_vegetation_parameters(
    file_path="./sample_data/vegetation_example_wide.csv",
    list_of_parameters=None,
    soil_parameters=soil_params,
    stand_parameters=stand_params,
    modeling_options=modeling_options_dict,
    sep=",",
)

frac_leaf_sym' set to 0.4
Available water capacity @Tlp (Campbell):280.7301527422195 mm
Available water capacity @P50 (Campbell):283.4407203700769 mm


/workspaces/sureau_ecos_py/sureau_ecos_py/plant_utils.py:899: UserWarning: Foliage evergreen has no params
  warnings.warn("Foliage evergreen has no params")
/tmp/ipykernel_5496/914077307.py:82: UserWarning: Make sure that depth from soil_parameters is in meters
  warnings.warn("Make sure that depth from soil_parameters is in meters")


defaultdict(list,
            {'c_lapoinit': 1e-05,
             'c_sapoinit': 2e-05,
             'fcrit': 450,
             'foliage': 'evergreen',
             'jarvis_par': 0.006,
             'k': 0.5,
             'k_ssyminit': 0.26,
             'ldmc': 570,
             'lma': 106,
             'p12_gs': -2.07,
             'p50_vc_leaf': -3.4,
             'p50_vc_stem': -3.4,
             'p88_gs': -2.62,
             'pt_coeff': 1.14,
             'pifullturgor_leaf': -2.1,
             'pifullturgor_stem': -2.1,
             'psi_close': -2,
             'psi_start_closing': -0.5,
             'q10_1_gmin': 1.2,
             'q10_2_gmin': 4.8,
             'tphase_gmin': 37.5,
             'tbase': 3,
             'tgs_optim': 25,
             'tgs_sens': 17,
             'apofrac_leaf': 0.4,
             'apofrac_stem': 0.4,
             'betarootprofile': 0.97,
             'canopystorageparam': 1.5,
             'daystart': 55,
             'epsilonsym_leaf': 10,
       